In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import cross_validation
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats

In [ ]:
# df = pd.read_csv('/root/thomas/github/cv_research/thomas/full_pipeline/small_pen/dataset_predictions_complete.csv')
df = pd.read_csv('/root/data/small_pen_data_collection/dataset_13k_pairs.csv')

In [ ]:
features = ['Unnamed: 0', '23', '24', '26', '46', 'ground_truth', 'left_image_key']
df = df[features].copy(deep=True)

In [ ]:
df.columns

In [ ]:

print(df.shape)
#print(df.isna().sum())

new_df = df.drop(columns=[])
#new_df = df.drop(columns=['14', '24', '34', '45', '46', '47']) 
#new_df = new_df.drop(columns=['16', '26', '36', '56', '67']) # '46', 

print(new_df.isna().sum())

new_df = new_df.dropna(subset=new_df.columns[1:-2])

print(new_df.shape)


my_mean = new_df.mean()
#my_sd = np.std(new_df, axis=0)

#print(my_mean)

means = []
stds = []

my_columns = new_df.columns[1:-2]

print(my_columns)

norm_df = new_df.copy()

iqrs = {}
for x in my_columns:
    my_mean = new_df[x].median()
    my_std = new_df[x].std()
    my_iqr = stats.iqr(new_df[x])
    
    means.append(my_mean)
    stds.append(my_std)
    
    #print(my_mean)
    #print(my_std)
    #print((new_df[x] - my_mean) / my_std)
    
    my_row = new_df[x].copy()
    new_df[x] = my_row / my_iqr
    iqrs[x] = my_iqr
    norm_df[x] = (my_row - my_mean) / my_std
    
#norm_df

In [ ]:
pd.groupby(alok_new_df, 'ground_truth')['23'].mean()

In [ ]:
df.iloc[:, 1:-2].shape

In [ ]:
my_df = new_df.loc[:, new_df.columns[1]:new_df.columns[-3]]
my_norm_df = norm_df.loc[:, new_df.columns[1]:new_df.columns[-3]]

array_subset = (np.abs(my_norm_df) > 1.5).any(axis=1) == False

print('Keeping %i of %i' % (np.sum(array_subset), my_norm_df.shape[0]))

my_df = my_df.loc[array_subset, :]

#Y = new_df['ground_truth']
Y = new_df['ground_truth'][array_subset]
print(my_df.shape)
#my_df_X = np.hstack((my_df, my_df ** 2, my_df ** 3))
pidx = np.indices((my_df.shape[1], my_df.shape[1])).reshape(2, -1)
lcol = pd.MultiIndex.from_product([my_df.columns, my_df.columns],
                                  names=[my_df.columns.name, my_df.columns.name])
my_df_X = pd.DataFrame(my_df.values[:, pidx[0]] * my_df.values[:, pidx[1]],
             columns=lcol)
print(my_df_X.shape)
#my_df_X = np.hstack(( my_df_X))

# my_df_X = sm.add_constant(my_df_X)
print(my_df_X.shape)              

In [ ]:
alok_new_df = new_df.loc[array_subset, :]

In [ ]:
pd.groupby(alok_new_df, 'ground_truth').count()

In [ ]:
pidx.shape

In [ ]:
lcol.shape

In [ ]:
my_df_X.shape

In [ ]:
pca = PCA(n_components=6)

pca.fit(my_df_X)
newX = pca.transform(my_df_X)
# new_X_compare = pca.fit_transform(my_df_X)

In [ ]:
newX

In [ ]:
plt.scatter(newX[:, 0], newX[:, 1])
plt.show()

In [ ]:
eigenvalues = pca.explained_variance_ratio_
print(np.sum(eigenvalues))

In [ ]:
components = pca.components_

newX = np.dot(my_df_X, components.T)

print(components.shape)

newX

#outlierIndices = np.where(newX[:,0] > 10)
#my_indices = my_df.index[outlierIndices[0]]


#my_df.loc[my_indices,:]

In [ ]:
newX.shape

In [ ]:
plt.scatter(newX[:,0], Y)#plt.scatter(np.log(newX[:,0]),np.log(Y))
plt.show()

In [ ]:
#myX = newX[:, 0]
myX = newX
myY = Y

plt.scatter(newX[:, 0], Y)
plt.show()

# myX = sm.add_constant(myX)

print(myX.shape)
print(myY.shape)

model = sm.OLS(myY, myX).fit()
predictions = model.predict(myX) # make the predictions by the model

print(model.params.shape)
#print(model.summary())

predY = predictions
#predY = np.exp(predictions)

#model.summary()

error = predY - myY

plt.scatter(Y, predY)

res = model.resid
fig = sm.qqplot(res, fit=True, line='45')

plt.show()

print('Error: %0.2f' % (np.median(np.abs(error)), ))
print('Pct Error: %0.2f' % (np.median(np.abs(error) / myY * 100), ))


In [ ]:
model.save('/root/data/models/biomass/model.pickle')

In [ ]:
np.save('/root/data/models/biomass/components.npy', components)

In [ ]:
components.shape

In [ ]:
from collections import Counter

In [ ]:
Counter(list(Y)).most_common()

In [ ]:
def cross_validation_per_fish(dataframe, Y, test_size = 0.2):
    nsamples = len(list(Y))
    most_common_list = Counter(list(Y)).most_common()
    
    np.random.shuffle(most_common_list)
    
    train_fish_ids = []
    test_fish_ids = []
    train_counter = 0
    for mc in most_common_list:
        train_counter += mc[1]
        if train_counter < nsamples * (1 - test_size):
            train_fish_ids.append(mc[0])
        else:
            test_fish_ids.append(mc[0])
    # create the datasets
    train_index = []
    test_index = []
    for (i, v) in enumerate(Y):
        if v in train_fish_ids:
            train_index.append(i)
        else:
            test_index.append(i)
#     print(train_index)
#     print(test_index)
    X_train = dataframe.iloc[train_index, :]
    Y_train = Y.iloc[train_index]
    X_test = dataframe.iloc[test_index, :]
    Y_test = Y.iloc[test_index]
    return X_train, X_test, Y_train, Y_test

In [ ]:
X_train, X_test, Y_train, Y_test = cross_validation_per_fish(my_df_X, Y)

In [ ]:
def getError(n_components, k):
    errors = []
    avg_errors = []
    avg_errors_raw = []
    error_pcts = []
    
    for i in range(k):
        # X_train, X_test, y_train, y_test = cross_validation.train_test_split(my_df_X, Y, test_size=0.4)
        X_train, X_test, y_train, y_test = cross_validation_per_fish(my_df_X, Y)
        
        pca = PCA(n_components=n_components)
        
        pca.fit(X_train)
        
        components = pca.components_

        newX = np.dot(X_train, components.T)

        #newX = pca.fit_transform(X_train)
        model = sm.OLS(y_train, newX).fit()
        
        newX_test = np.dot(X_test, components.T)
        #X_test = pca.transform(X_test)
        
        predY = model.predict(newX_test)
        error = predY - y_test

        errors.append(np.median(np.abs(error)))
        avg_errors.append(np.abs(np.mean(error)) / np.mean(y_test) * 100)
        avg_errors_raw.append(np.mean(error) / np.mean(y_test) * 100)
        error_pcts.append(np.median(np.abs(error) / y_test * 100))
    
    return (np.mean(errors), np.mean(avg_errors), avg_errors_raw, np.mean(error_pcts))

In [ ]:
#n_eigens = [5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 90, 100]
n_eigens = range(1, 10)

errors = []
avg_errors_raws = []

for n_eigen in n_eigens:
    myError = getError(n_eigen, 50)
    errors.append(myError[3])
    avg_errors_raws.append(myError[2])    
    #print(myError)
    print('Achieve %0.2f with %i eigenvectors' % (myError[1], n_eigen))
    
#print(avg_errors_raws[1])
plt.plot(avg_errors_raws[1])
plt.show()

print(.4 * newX.shape[0])

# plt.plot(n_eigens, errors)
# plt.xlabel('Number of regressors')
# plt.ylabel('Median error %')
# plt.show()


In [ ]:
'''
Extensions to this
- Try different filtering based off of total norm of covariance of eigenvectors
- More data
'''